> *Data import and Preprocessing*

In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_table('DLBCL.tab')
data.head()

,A28102,AB000114_at,AB000115_at,AB000220_at,AB000409_at,AB000449_at,AB000450_at,AB000460_at,AB000462_at,AB000464_at,AB000466_at,AB000467_at,AB000468_at,AB000584_at,AB000895_at,AB000896_at,AB000897_at,AB000905_at,AB001106_at,AB001325_at,AB002314_at,AB002315_at,AB002318_at,AB002365_at,AB002366_at,AB002380_at,AB002382_at,AB002409_at,AB002559_at,AB003102_at,AB003103_at,AB003177_at,AB003698_at,AB004884_at,AB006190_at,AC000061_cds2_at,AC000061_cds3_at,AC000062_at,AC000064_cds1_at,AC000064_cds2_at,...,X64177_f_at,X67491_f_at,X71345_f_at,X97444_f_at,Z80780_f_at,X00351_f_at,X01677_f_at,M31667_f_at,L41268_f_at,X99479_f_at,HG658-HT658_f_at,M94880_f_at,S80905_f_at,X03068_f_at,Z34822_f_at,U87593_f_at,U88902_cds1_f_at,AC002076_cds2_at,D64015_at,HG2510-HT2606_at,L10717_at,L34355_at,L78833_cds4_at,M13981_at,M21064_at,M93143_at,S78825_at,U11863_at,U29175_at,U48730_at,U58516_at,U73738_at,X06956_at,X16699_at,X83863_at,Z17240_at,L49218_f_at,M71243_f_at,Z78285_f_at,class
0,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,...,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,DLBCL FL
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,class
2,-1,-45,176,97,-57,233,265,945,56,819,-526,-1075,569,-501,-185,-28,-132,77,564,897,-40,169,1277,-80,-20,117,684,-6,500,864,109,838,332,762,548,368,-51,48,231,102,...,357,69,233,371,-92,21210,13993,1645,354,-341,12542,8233,298,11319,-72,229,778,16,409,41,111,234,354,71,286,575,-210,-1468,165,132,1036,-161,1523,-72,897,366,3,76,-28,DLBCL
3,25,-17,531,353,122,155,209,1688,42,639,-368,-253,892,71,157,-101,-68,136,562,623,4,278,956,193,17,344,768,226,390,912,223,904,270,668,619,248,-59,-18,153,73,...,7216,-27,333,287,-31,7490,15232,411,272,258,17542,14754,270,5700,-136,48,346,13,338,2,708,185,363,221,176,409,-262,-2333,2076,147,4254,-75,6056,-68,737,81,-4,41,-45,DLBCL
4,73,91,257,80,614,507,760,2252,196,863,-618,-596,1963,-226,-237,-133,29,116,999,861,55,539,1245,44,-2,224,597,275,1270,1976,320,1171,479,986,762,412,-65,22,424,115,...,5885,84,237,400,281,24760,14082,384,233,28,13338,15725,433,9175,-273,96,335,-31,318,35,453,112,263,266,299,630,-399,-2024,1468,169,1934,-129,2901,-9,946,450,38,164,-62,DLBCL


In [4]:
data = data.drop(data.index[[0,1]])
data.head()

,A28102,AB000114_at,AB000115_at,AB000220_at,AB000409_at,AB000449_at,AB000450_at,AB000460_at,AB000462_at,AB000464_at,AB000466_at,AB000467_at,AB000468_at,AB000584_at,AB000895_at,AB000896_at,AB000897_at,AB000905_at,AB001106_at,AB001325_at,AB002314_at,AB002315_at,AB002318_at,AB002365_at,AB002366_at,AB002380_at,AB002382_at,AB002409_at,AB002559_at,AB003102_at,AB003103_at,AB003177_at,AB003698_at,AB004884_at,AB006190_at,AC000061_cds2_at,AC000061_cds3_at,AC000062_at,AC000064_cds1_at,AC000064_cds2_at,...,X64177_f_at,X67491_f_at,X71345_f_at,X97444_f_at,Z80780_f_at,X00351_f_at,X01677_f_at,M31667_f_at,L41268_f_at,X99479_f_at,HG658-HT658_f_at,M94880_f_at,S80905_f_at,X03068_f_at,Z34822_f_at,U87593_f_at,U88902_cds1_f_at,AC002076_cds2_at,D64015_at,HG2510-HT2606_at,L10717_at,L34355_at,L78833_cds4_at,M13981_at,M21064_at,M93143_at,S78825_at,U11863_at,U29175_at,U48730_at,U58516_at,U73738_at,X06956_at,X16699_at,X83863_at,Z17240_at,L49218_f_at,M71243_f_at,Z78285_f_at,class
2,-1,-45,176,97,-57,233,265,945,56,819,-526,-1075,569,-501,-185,-28,-132,77,564,897,-40,169,1277,-80,-20,117,684,-6,500,864,109,838,332,762,548,368,-51,48,231,102,...,357,69,233,371,-92,21210,13993,1645,354,-341,12542,8233,298,11319,-72,229,778,16,409,41,111,234,354,71,286,575,-210,-1468,165,132,1036,-161,1523,-72,897,366,3,76,-28,DLBCL
3,25,-17,531,353,122,155,209,1688,42,639,-368,-253,892,71,157,-101,-68,136,562,623,4,278,956,193,17,344,768,226,390,912,223,904,270,668,619,248,-59,-18,153,73,...,7216,-27,333,287,-31,7490,15232,411,272,258,17542,14754,270,5700,-136,48,346,13,338,2,708,185,363,221,176,409,-262,-2333,2076,147,4254,-75,6056,-68,737,81,-4,41,-45,DLBCL
4,73,91,257,80,614,507,760,2252,196,863,-618,-596,1963,-226,-237,-133,29,116,999,861,55,539,1245,44,-2,224,597,275,1270,1976,320,1171,479,986,762,412,-65,22,424,115,...,5885,84,237,400,281,24760,14082,384,233,28,13338,15725,433,9175,-273,96,335,-31,318,35,453,112,263,266,299,630,-399,-2024,1468,169,1934,-129,2901,-9,946,450,38,164,-62,DLBCL
5,267,41,202,138,198,355,245,1469,170,384,-698,-31,1150,202,-541,162,-55,113,549,434,-51,418,900,47,15,226,430,89,476,1164,192,517,287,634,674,404,-122,21,154,174,...,2606,59,196,188,61,18747,17627,347,206,67,16591,9551,160,8049,-214,33,269,-23,224,1,335,-85,93,126,63,166,-99,-626,910,13,2469,93,2465,-119,339,321,-63,73,-13,DLBCL
6,16,24,187,39,145,254,571,930,-11,439,-665,-306,1460,117,-679,-61,-82,110,838,651,-74,201,454,0,-37,698,390,-17,461,815,342,444,345,648,587,198,-102,6,269,151,...,1408,-1,226,226,402,7871,14335,727,203,66,12369,5489,307,7405,-134,177,335,-8,248,24,170,-24,146,370,188,822,-29,-2380,350,69,608,-39,1667,-87,721,213,15,155,-26,DLBCL


In [5]:
data.index=range(0,len(data),1)
data.head()

,A28102,AB000114_at,AB000115_at,AB000220_at,AB000409_at,AB000449_at,AB000450_at,AB000460_at,AB000462_at,AB000464_at,AB000466_at,AB000467_at,AB000468_at,AB000584_at,AB000895_at,AB000896_at,AB000897_at,AB000905_at,AB001106_at,AB001325_at,AB002314_at,AB002315_at,AB002318_at,AB002365_at,AB002366_at,AB002380_at,AB002382_at,AB002409_at,AB002559_at,AB003102_at,AB003103_at,AB003177_at,AB003698_at,AB004884_at,AB006190_at,AC000061_cds2_at,AC000061_cds3_at,AC000062_at,AC000064_cds1_at,AC000064_cds2_at,...,X64177_f_at,X67491_f_at,X71345_f_at,X97444_f_at,Z80780_f_at,X00351_f_at,X01677_f_at,M31667_f_at,L41268_f_at,X99479_f_at,HG658-HT658_f_at,M94880_f_at,S80905_f_at,X03068_f_at,Z34822_f_at,U87593_f_at,U88902_cds1_f_at,AC002076_cds2_at,D64015_at,HG2510-HT2606_at,L10717_at,L34355_at,L78833_cds4_at,M13981_at,M21064_at,M93143_at,S78825_at,U11863_at,U29175_at,U48730_at,U58516_at,U73738_at,X06956_at,X16699_at,X83863_at,Z17240_at,L49218_f_at,M71243_f_at,Z78285_f_at,class
0,-1,-45,176,97,-57,233,265,945,56,819,-526,-1075,569,-501,-185,-28,-132,77,564,897,-40,169,1277,-80,-20,117,684,-6,500,864,109,838,332,762,548,368,-51,48,231,102,...,357,69,233,371,-92,21210,13993,1645,354,-341,12542,8233,298,11319,-72,229,778,16,409,41,111,234,354,71,286,575,-210,-1468,165,132,1036,-161,1523,-72,897,366,3,76,-28,DLBCL
1,25,-17,531,353,122,155,209,1688,42,639,-368,-253,892,71,157,-101,-68,136,562,623,4,278,956,193,17,344,768,226,390,912,223,904,270,668,619,248,-59,-18,153,73,...,7216,-27,333,287,-31,7490,15232,411,272,258,17542,14754,270,5700,-136,48,346,13,338,2,708,185,363,221,176,409,-262,-2333,2076,147,4254,-75,6056,-68,737,81,-4,41,-45,DLBCL
2,73,91,257,80,614,507,760,2252,196,863,-618,-596,1963,-226,-237,-133,29,116,999,861,55,539,1245,44,-2,224,597,275,1270,1976,320,1171,479,986,762,412,-65,22,424,115,...,5885,84,237,400,281,24760,14082,384,233,28,13338,15725,433,9175,-273,96,335,-31,318,35,453,112,263,266,299,630,-399,-2024,1468,169,1934,-129,2901,-9,946,450,38,164,-62,DLBCL
3,267,41,202,138,198,355,245,1469,170,384,-698,-31,1150,202,-541,162,-55,113,549,434,-51,418,900,47,15,226,430,89,476,1164,192,517,287,634,674,404,-122,21,154,174,...,2606,59,196,188,61,18747,17627,347,206,67,16591,9551,160,8049,-214,33,269,-23,224,1,335,-85,93,126,63,166,-99,-626,910,13,2469,93,2465,-119,339,321,-63,73,-13,DLBCL
4,16,24,187,39,145,254,571,930,-11,439,-665,-306,1460,117,-679,-61,-82,110,838,651,-74,201,454,0,-37,698,390,-17,461,815,342,444,345,648,587,198,-102,6,269,151,...,1408,-1,226,226,402,7871,14335,727,203,66,12369,5489,307,7405,-134,177,335,-8,248,24,170,-24,146,370,188,822,-29,-2380,350,69,608,-39,1667,-87,721,213,15,155,-26,DLBCL


In [6]:
data = data.rename(columns={'class':'Class'})
data.Class.value_counts()

DLBCL    58
FL       19
Name: Class, dtype: int64

In [7]:
dict = {
    'Class':{
        'DLBCL':0,
        'FL':1
    }
}   
data=data.replace(dict)
data.head()


,A28102,AB000114_at,AB000115_at,AB000220_at,AB000409_at,AB000449_at,AB000450_at,AB000460_at,AB000462_at,AB000464_at,AB000466_at,AB000467_at,AB000468_at,AB000584_at,AB000895_at,AB000896_at,AB000897_at,AB000905_at,AB001106_at,AB001325_at,AB002314_at,AB002315_at,AB002318_at,AB002365_at,AB002366_at,AB002380_at,AB002382_at,AB002409_at,AB002559_at,AB003102_at,AB003103_at,AB003177_at,AB003698_at,AB004884_at,AB006190_at,AC000061_cds2_at,AC000061_cds3_at,AC000062_at,AC000064_cds1_at,AC000064_cds2_at,...,X64177_f_at,X67491_f_at,X71345_f_at,X97444_f_at,Z80780_f_at,X00351_f_at,X01677_f_at,M31667_f_at,L41268_f_at,X99479_f_at,HG658-HT658_f_at,M94880_f_at,S80905_f_at,X03068_f_at,Z34822_f_at,U87593_f_at,U88902_cds1_f_at,AC002076_cds2_at,D64015_at,HG2510-HT2606_at,L10717_at,L34355_at,L78833_cds4_at,M13981_at,M21064_at,M93143_at,S78825_at,U11863_at,U29175_at,U48730_at,U58516_at,U73738_at,X06956_at,X16699_at,X83863_at,Z17240_at,L49218_f_at,M71243_f_at,Z78285_f_at,Class
0,-1,-45,176,97,-57,233,265,945,56,819,-526,-1075,569,-501,-185,-28,-132,77,564,897,-40,169,1277,-80,-20,117,684,-6,500,864,109,838,332,762,548,368,-51,48,231,102,...,357,69,233,371,-92,21210,13993,1645,354,-341,12542,8233,298,11319,-72,229,778,16,409,41,111,234,354,71,286,575,-210,-1468,165,132,1036,-161,1523,-72,897,366,3,76,-28,0
1,25,-17,531,353,122,155,209,1688,42,639,-368,-253,892,71,157,-101,-68,136,562,623,4,278,956,193,17,344,768,226,390,912,223,904,270,668,619,248,-59,-18,153,73,...,7216,-27,333,287,-31,7490,15232,411,272,258,17542,14754,270,5700,-136,48,346,13,338,2,708,185,363,221,176,409,-262,-2333,2076,147,4254,-75,6056,-68,737,81,-4,41,-45,0
2,73,91,257,80,614,507,760,2252,196,863,-618,-596,1963,-226,-237,-133,29,116,999,861,55,539,1245,44,-2,224,597,275,1270,1976,320,1171,479,986,762,412,-65,22,424,115,...,5885,84,237,400,281,24760,14082,384,233,28,13338,15725,433,9175,-273,96,335,-31,318,35,453,112,263,266,299,630,-399,-2024,1468,169,1934,-129,2901,-9,946,450,38,164,-62,0
3,267,41,202,138,198,355,245,1469,170,384,-698,-31,1150,202,-541,162,-55,113,549,434,-51,418,900,47,15,226,430,89,476,1164,192,517,287,634,674,404,-122,21,154,174,...,2606,59,196,188,61,18747,17627,347,206,67,16591,9551,160,8049,-214,33,269,-23,224,1,335,-85,93,126,63,166,-99,-626,910,13,2469,93,2465,-119,339,321,-63,73,-13,0
4,16,24,187,39,145,254,571,930,-11,439,-665,-306,1460,117,-679,-61,-82,110,838,651,-74,201,454,0,-37,698,390,-17,461,815,342,444,345,648,587,198,-102,6,269,151,...,1408,-1,226,226,402,7871,14335,727,203,66,12369,5489,307,7405,-134,177,335,-8,248,24,170,-24,146,370,188,822,-29,-2380,350,69,608,-39,1667,-87,721,213,15,155,-26,0


> *Scaling Data*

In [8]:
from sklearn import preprocessing
stdscaler = preprocessing.StandardScaler()
data1=pd.DataFrame(stdscaler.fit_transform(data),columns = data.columns)
scaler = preprocessing.MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(data1),columns = data.columns)

> *Running 10 fold CV without any filter*

In [9]:
from numpy import mean
from numpy import std
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
X = df_scaled.drop(['Class'], axis=1)
y = df_scaled.Class
cv = KFold(n_splits=10, random_state=1, shuffle=True)
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
RFC = RandomForestClassifier()
KNN=KNeighborsClassifier()
NB=GaussianNB()
scores = cross_val_score(RFC, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('RFC Accuracy 10 fold: %.4f (%.3f)' % (mean(scores)*100, std(scores)))
scores = cross_val_score(KNN, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('KNN Accuracy 10 fold: %.4f (%.3f)' % (mean(scores)*100, std(scores)))
scores = cross_val_score(NB, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Naive Bayes Accuracy 10 fold: %.4f (%.3f)' % (mean(scores)*100, std(scores)))


RFC Accuracy 10 fold: 88.2143 (0.123)
KNN Accuracy 10 fold: 88.0357 (0.118)
Naive Bayes Accuracy 10 fold: 81.6071 (0.139)


**Filter Ensemble**`

> *Chi-Square Test*

In [11]:
import scipy.stats as stats
def chi_sqr(test_column,output_column):
    observed_value = pd.crosstab(test_column,output_column)
    val = stats.chi2_contingency(observed_value)
    expected_value=val[3]
    for o,e in zip(observed_value.values,expected_value):
        chi_2 = sum([(o-e)**2./e])       
    chi2_stat = np.sum(chi_2)
    return chi2_stat

In [12]:
df1 = df_scaled.drop(['Class'], axis=1)
top_n1 =[]
for i in df1.columns.values:
    top_n1.append(chi_sqr(df1[i],df_scaled['Class']))    
col_name1 = np.array(X.columns)
a1 = pd.DataFrame(top_n1)
b1 = pd.DataFrame(col_name1)
info1 = pd.concat([a1,b1], axis=1)
info1.columns = ['Score','Features']
top1 = info1.nsmallest(50,'Score')
ft1 = np.array(top1['Features'])
top1
ft1

array(['AB000114_at', 'AB000115_at', 'AB000220_at', 'AB000409_at',
       'AB000449_at', 'AB000462_at', 'AB000466_at', 'AB000467_at',
       'AB000468_at', 'AB000895_at', 'AB000896_at', 'AB000897_at',
       'AB001106_at', 'AB002315_at', 'AB002318_at', 'AB002366_at',
       'AB002380_at', 'AB002382_at', 'AB002559_at', 'AB003102_at',
       'AB003103_at', 'AB003177_at', 'AB003698_at', 'AC000061_cds2_at',
       'AC000064_cds1_at', 'AC000066_at', 'AC000099_at',
       'AC000115_cds1_at', 'AC002086_at', 'AC002115_cds1_at',
       'AC002115_cds4_at', 'AC002115_rna2_at', 'AC002450_at',
       'AC002464_at', 'AC002486_at', 'AD000092_cds2_at',
       'AD000684_cds1_at', 'AD001527_cds1_at', 'AF000177_at',
       'AF000231_at', 'AF000234_at', 'AF000430_at', 'AF000560_at',
       'AF000573_rna1_at', 'AF000959_at', 'AF001620_at', 'AF002020_at',
       'AF003743_at', 'AF005043_at', 'AF005361_at'], dtype=object)

> *Mutual Information*

In [13]:
from sklearn.feature_selection import mutual_info_classif
X = df_scaled.drop(['Class'], axis=1)
y = df_scaled.Class
res = mutual_info_classif(X,y,discrete_features=True)
col_name2 = np.array(X.columns)
a2 = pd.DataFrame(res)
b2 = pd.DataFrame(col_name2)
info2 = pd.concat([a2,b2], axis=1)
info2.columns = ['Info_Gain','Features']
top2 = info2.nlargest(50,'Info_Gain')
ft2 = np.array(top2['Features'])
ft2

array(['D28483_at', 'D30037_at', 'D31886_at', 'D38500_at', 'D38553_at',
       'D49818_at', 'D50925_at', 'D50927_at', 'D83243_at', 'D89289_at',
       'HG1148-HT1148_at', 'HG172-HT3924_at', 'HG2662-HT2758_at',
       'L05568_at', 'L19871_at', 'L36463_at', 'L40394_at', 'M30496_at',
       'M31165_at', 'M33518_at', 'M54951_at', 'M55172_at', 'M58285_at',
       'M75110_at', 'M85164_at', 'S74221_at', 'S85963_at', 'U03399_at',
       'U07223_at', 'U09411_at', 'U09759_at', 'U15655_at', 'U31382_at',
       'U31449_at', 'U33839_at', 'U34038_at', 'U35139_at', 'U38847_at',
       'U48213_at', 'U50531_at', 'U52513_at', 'U52682_at', 'U63289_at',
       'U66059_cds7_at', 'U67171_at', 'U68142_at', 'U68536_at',
       'U77827_at', 'U80811_at', 'U86136_at'], dtype=object)

> *ReliefF*

In [15]:
pip install skrebate

  Created wheel for skrebate: filename=skrebate-0.62-cp37-none-any.whl size=29256 sha256=41c8e74f90795c28af300a66b63c8733a10e1914a3b86c15b5d9fc6ca9671aff
  Stored in directory: /root/.cache/pip/wheels/6b/0e/2f/540392932d3a01060f04c242d3b9bee80dfd87f63d80b3a9a0
Successfully built skrebate


In [16]:
from sklearn.pipeline import make_pipeline
from skrebate import ReliefF
from sklearn.pipeline import make_pipeline
from skrebate import ReliefF
features, classes = df_scaled.drop('Class', axis=1).values, df_scaled['Class'].values
arr = features.astype('float64')
fs = ReliefF()
fs.fit(arr, classes)
top_n3=[]
names=[]
for feature_name, feature_score in zip(df_scaled.drop('Class', axis=1).columns, fs.feature_importances_):
    top_n3.append(feature_score)
    names.append(feature_name)    
a3 = pd.DataFrame(top_n3)
b3 =pd.DataFrame(names)
info3 = pd.concat([a3,b3], axis=1)
info3.columns = ['Score','Features'] 
top3 = info3.nlargest(50,'Score')
ft3 = np.array(top3['Features'])
ft3


array(['M14328_s_at', 'X02152_at', 'AB002409_at', 'X16983_at',
       'L19686_rna1_at', 'Z11793_at', 'L42324_at', 'D87119_at',
       'Z30426_at', 'J04988_at', 'X12447_at', 'M63379_at', 'Z70723_at',
       'S73591_at', 'X52851_rna1_at', 'L17131_rna1_at', 'J03909_at',
       'HG2279-HT2375_at', 'J02645_at', 'J03626_rna1_s_at', 'M57703_s_at',
       'Z35227_at', 'AC002073_cds1_at', 'M10050_at', 'U16307_at',
       'X56494_at', 'Z21966_at', 'Z50781_at', 'M59829_at', 'L05187_at',
       'M37815_cds1_at', 'M99701_at', 'M22382_at', 'M12759_at',
       'HG417-HT417_s_at', 'U69108_at', 'J04173_at', 'M82827_s_at',
       'Z49269_at', 'S81957_at', 'L04282_at', 'U20647_at', 'Z26634_at',
       'U56814_at', 'U46006_s_at', 'M63138_at', 'D82348_at', 'L42621_at',
       'L04510_at', 'X86098_at'], dtype=object)

> *Union Function*

In [17]:
from functools import reduce
def top_fs_union(n):
    f1 = ft1[:n]
    f2 = ft2[:n]
    f3 = ft3[:n]
    return(reduce(np.union1d, (f1,f2,f3)))

> *Union function call with selected top n*

In [18]:
def assemble_run(n,classifier):
  top_fs=top_fs_union(n)
  top_fs=np.append(top_fs,['Class'], axis=0)
  dfs = df_scaled.loc[:,top_fs]
  X = dfs.drop(['Class'], axis=1)
  y = dfs['Class']
  model = classifier()
  scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  print('Accuracy 10 fold: %.4f (%.3f)' % (mean(scores)*100, std(scores)))

**Accuracy**

> *RANDOM FOREST CLASSIFIER*

In [19]:
#Feature Subset by Union of Best Features from each

RFC1 = assemble_run(5,RandomForestClassifier)
RFC1 = assemble_run(10,RandomForestClassifier)
RFC1 = assemble_run(15,RandomForestClassifier)
RFC1 = assemble_run(20,RandomForestClassifier)
RFC1 = assemble_run(25,RandomForestClassifier)
RFC1 = assemble_run(30,RandomForestClassifier)
RFC1 = assemble_run(35,RandomForestClassifier)
RFC1 = assemble_run(40,RandomForestClassifier)
RFC1 = assemble_run(45,RandomForestClassifier)
RFC1 = assemble_run(50,RandomForestClassifier)


Accuracy 10 fold: 89.1071 (0.150)
Accuracy 10 fold: 93.3929 (0.066)
Accuracy 10 fold: 97.3214 (0.054)
Accuracy 10 fold: 91.9643 (0.092)
Accuracy 10 fold: 91.9643 (0.092)
Accuracy 10 fold: 96.0714 (0.060)
Accuracy 10 fold: 91.9643 (0.092)
Accuracy 10 fold: 93.2143 (0.093)
Accuracy 10 fold: 91.9643 (0.092)
Accuracy 10 fold: 90.7143 (0.088)


In [20]:
#Since the best accuracy was in the case of 15 features

RFC1 = assemble_run(11,RandomForestClassifier)
RFC1 = assemble_run(12,RandomForestClassifier)
RFC1 = assemble_run(13,RandomForestClassifier)
RFC1 = assemble_run(14,RandomForestClassifier)
RFC1 = assemble_run(15,RandomForestClassifier)
RFC1 = assemble_run(16,RandomForestClassifier)
RFC1 = assemble_run(17,RandomForestClassifier)
RFC1 = assemble_run(18,RandomForestClassifier)


Accuracy 10 fold: 93.3929 (0.066)
Accuracy 10 fold: 93.2143 (0.093)
Accuracy 10 fold: 94.8214 (0.064)
Accuracy 10 fold: 96.0714 (0.060)
Accuracy 10 fold: 96.0714 (0.060)
Accuracy 10 fold: 93.2143 (0.130)
Accuracy 10 fold: 91.7857 (0.130)
Accuracy 10 fold: 93.2143 (0.093)


> *KNN CLASSIFIER*

In [21]:
#Feature Subset by Union of Best Features from each

KNN1 = assemble_run(5,KNeighborsClassifier)
KNN1 = assemble_run(10,KNeighborsClassifier)
KNN1 = assemble_run(15,KNeighborsClassifier)
KNN1 = assemble_run(20,KNeighborsClassifier)
KNN1 = assemble_run(25,KNeighborsClassifier)
KNN1 = assemble_run(30,KNeighborsClassifier)
KNN1 = assemble_run(35,KNeighborsClassifier)
KNN1 = assemble_run(40,KNeighborsClassifier)
KNN1 = assemble_run(45,KNeighborsClassifier)
KNN1 = assemble_run(50,KNeighborsClassifier)


Accuracy 10 fold: 89.2857 (0.105)
Accuracy 10 fold: 90.7143 (0.105)
Accuracy 10 fold: 93.5714 (0.085)
Accuracy 10 fold: 98.5714 (0.043)
Accuracy 10 fold: 97.3214 (0.054)
Accuracy 10 fold: 98.5714 (0.043)
Accuracy 10 fold: 97.3214 (0.054)
Accuracy 10 fold: 97.3214 (0.054)
Accuracy 10 fold: 97.3214 (0.054)
Accuracy 10 fold: 97.3214 (0.054)


> *NAIVE BAYES CLASSIFIER*

In [22]:
#Feature Subset by Union of Best Features from each

NB1 = assemble_run(5,GaussianNB)
NB1 = assemble_run(10,GaussianNB)
NB1 = assemble_run(15,GaussianNB)
NB1 = assemble_run(20,GaussianNB)
NB1 = assemble_run(25,GaussianNB)
NB1 = assemble_run(30,GaussianNB)
NB1 = assemble_run(35,GaussianNB)
NB1 = assemble_run(40,GaussianNB)
NB1 = assemble_run(45,GaussianNB)
NB1 = assemble_run(50,GaussianNB)


Accuracy 10 fold: 89.2857 (0.119)
Accuracy 10 fold: 88.0357 (0.114)
Accuracy 10 fold: 89.2857 (0.119)
Accuracy 10 fold: 89.2857 (0.105)
Accuracy 10 fold: 93.0357 (0.114)
Accuracy 10 fold: 90.7143 (0.105)
Accuracy 10 fold: 89.2857 (0.119)
Accuracy 10 fold: 89.2857 (0.119)
Accuracy 10 fold: 89.2857 (0.119)
Accuracy 10 fold: 89.2857 (0.119)


> *Accuracy check for individal feature set*

In [23]:
def fs_run(classifier,feature_set,no_of_attr):
  ft=feature_set[:no_of_attr]
  top_fs=np.append(ft,['Class'], axis=0)
  dfs = df_scaled.loc[:,top_fs]
  X = dfs.drop(['Class'], axis=1)
  y = dfs['Class']
  model = classifier()
  scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  print('Accuracy 10 fold: %.4f (%.3f)' % (mean(scores)*100, std(scores)))

In [24]:
print('Accuracy for Feature set 1 with best attributes:')
fs_run(RandomForestClassifier,ft1,15)
fs_run(KNeighborsClassifier,ft1,15)
fs_run(GaussianNB,ft1,15)
print('Accuracy for Feature set 2 with best attributes:')
fs_run(RandomForestClassifier,ft2,15)
fs_run(KNeighborsClassifier,ft2,15)
fs_run(GaussianNB,ft2,15)
print('Accuracy for Feature set 3 with best attributes :')
fs_run(RandomForestClassifier,ft3,15)
fs_run(KNeighborsClassifier,ft3,15)
fs_run(GaussianNB,ft3,15)

Accuracy for Feature set 1 with best attributes:
Accuracy 10 fold: 74.1071 (0.147)
Accuracy 10 fold: 79.2857 (0.160)
Accuracy 10 fold: 72.5000 (0.153)
Accuracy for Feature set 2 with best attributes:
Accuracy 10 fold: 80.5357 (0.116)
Accuracy 10 fold: 72.5000 (0.172)
Accuracy 10 fold: 68.9286 (0.151)
Accuracy for Feature set 3 with best attributes :
Accuracy 10 fold: 94.6429 (0.066)
Accuracy 10 fold: 94.4643 (0.093)
Accuracy 10 fold: 94.4643 (0.093)
